# Poly-Yolo for steel defect detection
### This [model](http://https://gitlab.com/irafm-ai/poly-yolo/-/tree/master/poly_yolo) can be used in objective detection and instance segmentation
Thanks IRAFM AI for creating this work, and also thanks the following notebooks as references.

https://www.kaggle.com/go1dfish/clear-mask-visualization-and-simple-eda/notebook

https://www.kaggle.com/rishabhiitbhu/unet-starter-kernel-pytorch-lb-0-88

Segmentation problem to objective detection problem.

## Load data and other settings

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### Custom files instruction
yolo_anchors.txt and *yolo_classes.txt* should be customized according to your dataset
The yolo_anchor can be calculated using K-means in the following code(using *kmeans.py*) and add *yolo_classes.txt* by oneself, for this competition, the classes are '1,2,3,4'.

In [ ]:
import sys
package_path = '../input/polyyolo/poly_yolo_simple/'
sys.path.append(package_path)
package_path = '../input/polyyolo/'
sys.path.append(package_path)
import poly_yolo_new as yolo
# poly_yolo_new is different from poly_yolo only in the use of multi-gpu, new version disable it becuase of keras version
input_dir = "../input/"

In [ ]:
import numpy as np # linear algebra
import pandas as pd
pd.set_option("display.max_rows", 101)
import os
print(os.listdir("../input"))
import cv2
import json
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["font.size"] = 15
import seaborn as sns
from collections import Counter
from PIL import Image
import math
import seaborn as sns
from collections import defaultdict
from pathlib import Path
import cv2
from tqdm import tqdm

In [ ]:
train_df = pd.read_csv("../input/severstal-steel-defect-detection/train.csv")
sample_df = pd.read_csv("../input/severstal-steel-defect-detection/sample_submission.csv")

### Preprocess training data and submission data to unify public&private board data

In [ ]:
train_original = train_df.copy()
def trans_train(train_df):
    train_format = []
    for indexs, content in train_df.iterrows():
        ImageId = content[0]
        ClassId = content[1]
        EncodedPixels = content[2]
        classes = [1,2,3,4]
        for class_ in classes:
            name = str(ImageId) + '_' + str(class_)
            val_name = str(ImageId) + '_' + str(ClassId)
            if name == val_name:
                pixel = EncodedPixels
            else: pixel = ''
            train_format.append([name, pixel])
    train_df_new = pd.DataFrame(train_format, columns=['ImageId_ClassId', 'EncodedPixels'])
    return train_df_new

if len(train_df.columns) == 3:
    train_df = trans_train(train_df)
else: print('continue')

## Check image data and visualize the masks

In [ ]:
train_size_dict = defaultdict(int)
train_path = Path("../input/severstal-steel-defect-detection/train_images/")

for img_name in train_path.iterdir():
    img = Image.open(img_name)
    train_size_dict[img.size] += 1
    
test_size_dict = defaultdict(int)
test_path = Path("../input/severstal-steel-defect-detection/test_images/")

for img_name in test_path.iterdir():
    img = Image.open(img_name)
    test_size_dict[img.size] += 1

In [ ]:
# color of masks
palette = [(249, 192, 12), (0, 185, 241), (114, 0, 218), (249,50,12)]
# anchor_mask size: all images are in this size
width = 1600
height = 256
classes_num = 4

# return the mask
def name_and_mask(start_idx, width, height, classes_num):
    col = start_idx
    img_names = [str(i).split("_")[0] for i in train_df.iloc[col:col+4, 0].values]
    if not (img_names[0] == img_names[1] == img_names[2] == img_names[3]):
        raise ValueError

    labels = train_df.iloc[col:col+4, 1]
    mask = np.zeros((height, width, classes_num), dtype=np.uint8)

    for idx, label in enumerate(labels.values):
        if label is not '':
            mask_label = np.zeros(width*height, dtype=np.uint8)
            label = label.split(" ")
            positions = map(int, label[0::2])
            length = map(int, label[1::2])
            for pos, le in zip(positions, length):
                mask_label[pos-1:pos+le-1] = 1
            mask[:, :, idx] = mask_label.reshape(height, width, order='F')
    return img_names[0], mask

def show_mask_image(col, width, height, classes_num):
    name, mask = name_and_mask(col,width, height, classes_num)
    img = cv2.imread(str(train_path / name))
    fig, ax = plt.subplots(figsize=(15, 15))
    # use 4 contours to category the 4 classes
    for ch in range(4):
        contours, _ = cv2.findContours(mask[:, :, ch], cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
        for i in range(0, len(contours)):
            cv2.polylines(img, contours[i], True, palette[ch], 2)
    ax.set_title(name)
    ax.imshow(img)
    plt.show()

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(15, 5))
for i in range(4):
    ax[i].axis('off')
    ax[i].imshow(np.ones((50, 50, 3), dtype=np.uint8) * palette[i])
    ax[i].set_title("class color: {}".format(i+1))
fig.suptitle("each class colors")

plt.show()

In [ ]:
idxs = [12,16,20,24,32]
for idx in idxs:
    show_mask_image(idx, width, height, classes_num)
    

### Using Kmeans to Create anchor files

#### From segmentation data to anchor data

In [ ]:
name, mask = name_and_mask(12,width, height, classes_num)
mask.shape

In [ ]:
# mask -> width * height, only one number to indicate class
# flat into one layer (according to the links above, we know that there is no case of overlap defects)

def mask_format(mask, width, height):
#     mask_format = np.zeros(width*height, dtype=np.uint8)
#     mask_format = mask_format.reshape(height, width, order='F')
#     for i in range(height):
#         for j in range(width):
#             mask_format[i][j] = sum(mask[i][j])
    mask_format1 = mask[:,:,0]
    mask_format2 = mask[:,:,1]
    mask_format3 = mask[:,:,2]
    mask_format4 = mask[:,:,3]
    return mask_format1, mask_format2, mask_format3, mask_format4

mask_format1,mask_format2,mask_format3,mask_format4  = mask_format(mask, width, height)

In [ ]:

# bbox = find_bbox(mask_format4)
# for item in bbox:
#     [x1,y1,x2,y2] = [item[0], item[1], item[0] + item[2], item[1] + item[3]]
   

In [ ]:
def find_bbox(mask):
    _, labels, stats, centroids = cv2.connectedComponentsWithStats(mask.astype(np.uint8))
    stats = stats[stats[:,4].argsort()]
    return stats[:-1]

def draw_box(mask_format):
    from  matplotlib import patches
    ax = plt.axes()
    plt.imshow(mask_format,cmap='bone')
    bboxs = find_bbox(mask_format)
    for j in bboxs: 
        rect = patches.Rectangle((j[0],j[1]),j[2],j[3],linewidth=1,edgecolor='r',facecolor='none')
        ax.add_patch(rect)
    plt.show() 

# find out what type of defect this image has
print('pixel number of 4 types of defects:')
print(sum(sum(mask_format1)),sum(sum(mask_format2)),sum(sum(mask_format3)),sum(sum(mask_format4)))

draw_box(mask_format4)
   

## Load pretrained model

In [ ]:
# model1 = yolo.YOLO(model_path='../input/polyyolo/poly_yolo.h5')